In [6]:
import pandas as pd
import numpy as np

df = pd.read_csv('../../MEGA/Data/Kinect trials/2014-12-22_P007_Pre_004.txt', skiprows=range(22),\
                header=None, names = [i for i in range(-2, 60)], sep='\t', engine='python')

# Change some column names
df.rename(columns={-2: 'Frame', -1: 'Part'}, inplace=True)

In [7]:
df_HEAD = df[df['Part'] == 'HEAD']

In [8]:
# Replace any strings with nan in the Frame column
df['Frame'] = df['Frame'].replace(r'[^0-9]', np.nan, regex=True)

In [9]:
# Convert the strings in the frame column to numbers
df['Frame'] = pd.to_numeric(df['Frame'])

In [10]:
n_frames= max(df['Frame'])

In [11]:
frame_624 = df[df['Frame'] == 624]

In [12]:
x = frame_624.as_matrix()

In [38]:
# Part names
parts = df.groupby('Part').groups.keys()

In [87]:
n_parts = len(parts)

# Dataframe for current image frame
frame_df = frame_624


In [176]:
max_num_coords = 60
part = 'R_HAND'

def read_positions(frame_df, part, max_num_coords):
    
    row = frame_df[frame_df['Part'] == part]
    row_coordinates = row.loc[:, range(max_num_coords)].as_matrix()

    n_coordinates = np.sum(~np.isnan(row_coordinates))
    n_points = int(n_coordinates / 3)
    
    points = np.full([n_points, 3], np.nan)

    for idx, i in enumerate(range(0, n_coordinates, 3)):
        points[idx, :] = row_coordinates[0, i:i+3]
        
    return points    
        
population_dict = {part: read_positions(frame_df, part, max_num_coords) for part in parts}

In [190]:
part_types = ['HEAD', 'HIP', 'UPPER_LEG', 'KNEE', 'LOWER_LEG', 'FOOT']

In [249]:

def get_population(population_dict, part_types):

    population_list, labels = [], []

    for part_type in part_types:
        for full_part_name in population_dict:

            if part_type in full_part_name:
                points = population_dict[full_part_name]
                n_points, _ = points.shape

                population_list.append(points)
                labels.extend([part_type for i in range(n_points)])

    # Convert list to numpy matrix            
    population = np.concatenate(population_list)

    assert(population.shape[0] == len(labels))
    
    return population, labels

population, labels = get_population(population_dict, part_types)